# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe muito. os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou m- Observeuito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

### 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático. 

In [ ]:
for coluna in df.columns[3:]:
    print(coluna, '\n', df[coluna].value_counts(), '\n')

In [ ]:
y, X = patsy.dmatrices(formula_like='''
                                    np.log(renda) ~ sexo 
                                                    + posse_de_veiculo 
                                                    + posse_de_imovel 
                                                    + C(qtd_filhos, Treatment(0))
                                                    + C(tipo_renda, Treatment('Assalariado'))
                                                    + C(educacao, Treatment('Secundário')) 
                                                    + C(estado_civil, Treatment('Casado')) 
                                                    + C(tipo_residencia, Treatment('Casa')) 
                                                    + idade 
                                                    + tempo_emprego 
                                                    + C(qt_pessoas_residencia, Treatment(2))
                                    ''', 
                       data=df)

In [ ]:
sm.OLS(y, X).fit().summary()

### 2. Remova a variável menos significante e analise:
     - Observe muito. os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
     - Observe os parâmetros e veja se algum se alterou m- Observeuito.

In [ ]:
smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel 
                                + C(qtd_filhos, Treatment(0))
                                + C(tipo_renda, Treatment('Assalariado'))
                                + C(educacao, Treatment('Secundário')) 
                                + C(estado_civil, Treatment('Casado')) 
                                + idade 
                                + tempo_emprego 
                                + C(qt_pessoas_residencia, Treatment(2))
                ''', 
        data=df).fit().summary()

**Resposta:**

Após remover a variável tipo_residencia, o modelo não apresentou melhora nem piora significativa. O coeficiente de determinação manteve o mesmo valor de 0.358 e o p-value das variáveis teve uma variação muito pequena, praticamente insignificante.

### 3. Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.


In [ ]:
smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel
                                + tipo_renda == 'Empresário'
                                + tipo_renda == 'Servidor público'
                                + educacao == 'Superior completo'
                                + qt_pessoas_residencia == 3
                                + qt_pessoas_residencia == 4
                                + idade 
                                + tempo_emprego
                ''', 
        data=df).fit().summary()

**Resposta:**

Ao comparar o modelo final com o modelo inicial, os indicadores não apresentaram nenhuma melhora ou piora significativa. O coeficiente de determinação diminuiu de 0.358 para 0.357 e o AIC de 27190 para 27170.